# MARCH MADNESS PROJECT - JAMAL ZLITNI

In [6]:
%matplotlib inline

import string
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import psycopg2
import seaborn as sbn
from altair import Chart, X, Y, Color, Scale
import altair as alt
from vega_datasets import data
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
matplotlib.style.use('ggplot')

# large thanks to this Kaggle database that started my research and was the beginning of the data below
# https://www.kaggle.com/datasets/nishaanamin/march-madness-data

In [7]:
# read in the csv files
winners = pd.read_csv('./winners.csv')
winners = winners.dropna()
winners.head()

first_round_upsets = pd.read_csv('./first_round_upsets.csv')
first_round_upsets = first_round_upsets.dropna()
first_round_upsets.head()

upsets = pd.read_csv('./Upset Count Data.csv')
upsets = upsets.dropna()
upsets.head()

conf_data = pd.read_csv('./Tournament Conference Data.csv')
conf_data = conf_data.dropna()
conf_data.head()

trends = pd.read_csv('./Tournament Trends.csv')
trends.head()

game_data = pd.read_csv('./Tournament Game Data.csv')
game_data = game_data.dropna()
game_data.head()

,YEAR,SEED,TEAM,TEAM ROUND,CURRENT ROUND,KENPOM ADJUSTED EFFICIENCY,KENPOM ADJUSTED OFFENSE,KENPOM ADJUSTED DEFENSE,KENPOM ADJUSTED TEMPO,BARTTORVIK ADJUSTED EFFICIENCY,...,OP D REB %,BLOCKED %,TURNOVER % DEFENSE,WINS ABOVE BUBBLE,WIN %,POINTS PER POSSESSION OFFENSE,POINTS PER POSSESSION DEFENSE,SCORE,ID,TEAM.1
0,2022,1,Gonzaga,16,64,33.0,121.8,88.8,72.5,31.8,...,71.0,6.6,17.0,6.7,89.66,1.190,0.885,93,1,Gonzaga
1,2022,16,Georgia St.,64,64,1.8,101.5,99.7,67.1,1.4,...,67.2,8.9,22.7,-5.2,60.00,0.993,0.978,72,2,Georgia St.
2,2022,8,Boise St.,64,64,16.4,108.4,92.0,64.6,15.2,...,69.4,8.9,20.1,3.5,78.79,1.038,0.928,53,3,Boise St.
3,2022,9,Memphis,32,64,16.2,110.4,94.1,70.3,17.5,...,62.5,9.1,20.9,1.4,67.74,1.056,0.957,64,4,Memphis
4,2022,5,Connecticut,64,64,19.3,113.9,94.6,64.9,18.6,...,62.0,8.3,18.0,3.2,71.88,1.099,0.953,63,5,Connecticut


-------------------------
# WINNERS ANALYSIS

In [8]:
print(winners.head())

   YEAR  SEED         TEAM  ROUND  KENPOM ADJUSTED EFFICIENCY  \
0  2023     4  Connecticut      1                        25.3   
1  2022     1       Kansas      1                        25.5   
2  2021     1       Baylor      1                        29.5   
3  2019     1    Virginia       1                        35.7   
4  2018     1    Villanova      1                        31.4   

   KENPOM ADJUSTED OFFENSE  KENPOM ADJUSTED DEFENSE  KENPOM ADJUSTED TEMPO  \
0                    119.5                     94.2                   66.6   
1                    119.4                     93.9                   69.1   
2                    123.3                     93.8                   68.4   
3                    123.6                     87.9                   59.3   
4                    127.4                     96.0                   68.3   

   BARTTORVIK ADJUSTED EFFICIENCY  BARTTORVIK ADJUSTED OFFENSE  ...  \
0                            25.7                        119.8  ...  

In [9]:
print("Here are the lowest values for the following statistics that any tournament winning team has had since" + 
      " 2008, DON'T pick a team to win that has statistics lower than these: \n")
print("KENPOM ADJUSTED EFFICIENCY: ", winners['KENPOM ADJUSTED EFFICIENCY'].min())
print("KENPOM ADJUSTED OFFENSE: ", winners['KENPOM ADJUSTED OFFENSE'].min())
print("KENPOM ADJUSTED DEFENSE: ", winners['KENPOM ADJUSTED DEFENSE'].min())
print("KENPOM ADJUSTED TEMPO: ", winners['KENPOM ADJUSTED TEMPO'].min())
print("BARTTORVIK ADJUSTED EFFICIENCY: ", winners['BARTTORVIK ADJUSTED EFFICIENCY'].min())
print("BARTTORVIK ADJUSTED OFFENSE: ", winners['BARTTORVIK ADJUSTED OFFENSE'].min())
print("2PT % DEFENSE: ", winners['2PT % DEFENSE'].min())
print("3PT % DEFENSE: ", winners['3PT % DEFENSE'].min())
print("TURNOVER % DEFENSE: ", winners['TURNOVER % DEFENSE'].min())
print("WIN %: ", winners['WIN %'].min())
print("POINTS PER POSSESSION OFFENSE: ", winners['POINTS PER POSSESSION OFFENSE'].min())

Here are the lowest values for the following statistics that any tournament winning team has had since 2008, DON'T pick a team to win that has statistics lower than these: 

KENPOM ADJUSTED EFFICIENCY:  19.1
KENPOM ADJUSTED OFFENSE:  111.4
KENPOM ADJUSTED DEFENSE:  83.2
KENPOM ADJUSTED TEMPO:  59.3
BARTTORVIK ADJUSTED EFFICIENCY:  19.0
BARTTORVIK ADJUSTED OFFENSE:  111.6
2PT % DEFENSE:  38.8
3PT % DEFENSE:  27.2
TURNOVER % DEFENSE:  17.8
WIN %:  74.29
POINTS PER POSSESSION OFFENSE:  1.085


In [10]:
print("When considering the odds of a specific seeding winning the tournament by going off of every prior March Madness tournament are as follows: \n")
print("1 seed: 64.9% \n2 seed: 13.5% \n3 seed: 10.8% \n4 seed: 2.7% \n5 seed: 0% \n6 seed: 2.7% \n7 seed: 2.7% \n8 seed: 2.7% \n9 seed and beyond: 0%")

When considering the odds of a specific seeding winning the tournament by going off of every prior March Madness tournament are as follows: 

1 seed: 64.9% 
2 seed: 13.5% 
3 seed: 10.8% 
4 seed: 2.7% 
5 seed: 0% 
6 seed: 2.7% 
7 seed: 2.7% 
8 seed: 2.7% 
9 seed and beyond: 0%


Some notable takeaways from above are that a 1 seed has won the tournament over half the time, a 2 and 3 seed have almost the same win percentage (a 2 seed has only won one more time than a 3 seed has won), and even though a 4, 6, 7, or 8 seed have all won the tournament once, a 5 seed has actually never won!

----------------------------------------------------------
# FIRST ROUND UPSET ANALYSIS

In [11]:
print(first_round_upsets)

    YEAR one v sixteen two v fifteen three v fourteen four v thirteen  \
0   2023             1             1                0               1   
1   2022             0             1                0               0   
2   2021             0             1                1               2   
3   2019             0             0                0               1   
4   2018             1             0                0               2   
5   2017             0             0                0               0   
6   2016             0             1                1               1   
7   2015             0             0                2               0   
8   2014             0             0                1               0   
9   2013             0             1                1               1   
10  2012             0             2                0               1   
11  2011             0             0                0               1   
12  2010             0             0               

In [12]:
first_round_upsets['one v sixteen'] = first_round_upsets['one v sixteen'].astype(int)
first_round_upsets['two v fifteen'] = first_round_upsets['two v fifteen'].astype(int)
first_round_upsets['three v fourteen'] = first_round_upsets['three v fourteen'].astype(int)
first_round_upsets['four v thirteen'] = first_round_upsets['four v thirteen'].astype(int)
first_round_upsets['five v twelve'] = first_round_upsets['five v twelve'].astype(int)
first_round_upsets['six v eleven'] = first_round_upsets['six v eleven'].astype(int)
first_round_upsets['seven v ten'] = first_round_upsets['seven v ten'].astype(int)

print("Using data from 2008 onwards: \n")
print("Here are the average number of first round upsets: ", first_round_upsets['TOTAL'].mean().round(2))
print("\nAverage number of first round upsets per year broken down by seeding matchup:")

print("1 vs 16: ", first_round_upsets['one v sixteen'].mean().round(2))
print("2 vs 15: ", first_round_upsets['two v fifteen'].mean().round(2))
print("3 vs 14: ", first_round_upsets['three v fourteen'].mean().round(2))
print("4 vs 13: ", first_round_upsets['four v thirteen'].mean().round(2))
print("5 vs 12: ", first_round_upsets['five v twelve'].mean().round(2))
print("6 vs 11: ", first_round_upsets['six v eleven'].mean().round(2))
print("7 vs 10: ", first_round_upsets['seven v ten'].mean().round(2))

print("\nHere are the percentage of years that have had at least one upset by seed matchup: ")
print("1 vs 16: ", (((first_round_upsets['one v sixteen'] > 0).sum() / len(first_round_upsets['one v sixteen'])) * 100).round(2))
print("2 vs 15: ", (((first_round_upsets['two v fifteen'] > 0).sum() / len(first_round_upsets['two v fifteen'])) * 100).round(2))
print("3 vs 14: ", (((first_round_upsets['three v fourteen'] > 0).sum() / len(first_round_upsets['three v fourteen'])) * 100).round(2))
print("4 vs 13: ", (((first_round_upsets['four v thirteen'] > 0).sum() / len(first_round_upsets['four v thirteen'])) * 100).round(2))
print("6 vs 11: ", (((first_round_upsets['six v eleven'] > 0).sum() / len(first_round_upsets['six v eleven'])) * 100).round(2))
print("7 vs 10: ", (((first_round_upsets['seven v ten'] > 0).sum() / len(first_round_upsets['seven v ten'])) * 100).round(2))

print("\nHere are the percentage of years that have had at least two upsets by seed matchup: ")
print("1 vs 16: ", (((first_round_upsets['one v sixteen'] > 1).sum() / len(first_round_upsets['one v sixteen'])) * 100).round(2))
print("2 vs 15: ", (((first_round_upsets['two v fifteen'] > 1).sum() / len(first_round_upsets['two v fifteen'])) * 100).round(2))
print("3 vs 14: ", (((first_round_upsets['three v fourteen'] > 1).sum() / len(first_round_upsets['three v fourteen'])) * 100).round(2))
print("4 vs 13: ", (((first_round_upsets['four v thirteen'] > 1).sum() / len(first_round_upsets['four v thirteen'])) * 100).round(2))
print("6 vs 11: ", (((first_round_upsets['six v eleven'] > 1).sum() / len(first_round_upsets['six v eleven'])) * 100).round(2))
print("7 vs 10: ", (((first_round_upsets['seven v ten'] > 1).sum() / len(first_round_upsets['seven v ten'])) * 100).round(2))

Using data from 2008 onwards: 

Here are the average number of first round upsets:  7.13

Average number of first round upsets per year broken down by seeding matchup:
1 vs 16:  0.13
2 vs 15:  0.47
3 vs 14:  0.47
4 vs 13:  0.93
5 vs 12:  1.6
6 vs 11:  1.93
7 vs 10:  1.6

Here are the percentage of years that have had at least one upset by seed matchup: 
1 vs 16:  13.33
2 vs 15:  40.0
3 vs 14:  40.0
4 vs 13:  73.33
6 vs 11:  100.0
7 vs 10:  100.0

Here are the percentage of years that have had at least two upsets by seed matchup: 
1 vs 16:  0.0
2 vs 15:  6.67
3 vs 14:  6.67
4 vs 13:  20.0
6 vs 11:  66.67
7 vs 10:  40.0


Takeaways
- This shows that 40% of the time there will be a 5/12 upset, and the same for 3/14. 
- In about 3 out of 4 years there will be a 4/13 upset
- There is an EXTREMELY LIKELY chance that a 6/11 and 7/10 upset will happen, there has been at least one of each in every tournament for the last 15 years.
- In fact, more often than not there are 2 or more 6/11 upsets in the same year than

-------------------------------------

# TOTAL UPSET ANALYSIS (seed differential of 3 or more)

In [13]:
upsets

,YEAR,FIRST ROUND,SECOND ROUND,SWEET 16,ELITE 8,FINAL 4,TOTAL
0,2023.0,5.0,4.0,4.0,2.0,0.0,15.0
1,2022.0,7.0,5.0,4.0,0.0,1.0,17.0
2,2021.0,9.0,6.0,2.0,1.0,0.0,18.0
3,2019.0,8.0,0.0,1.0,2.0,0.0,11.0
4,2018.0,6.0,5.0,3.0,1.0,0.0,15.0
5,2017.0,5.0,4.0,2.0,2.0,0.0,13.0
6,2016.0,10.0,2.0,0.0,1.0,0.0,13.0
7,2015.0,5.0,3.0,1.0,1.0,0.0,10.0
8,2014.0,7.0,5.0,3.0,2.0,2.0,19.0
9,2013.0,8.0,3.0,2.0,1.0,0.0,14.0


FROM 2008 ONWARDS:

In [14]:
print('The average first round upsets: ', upsets["FIRST ROUND"].mean().round(2))
print('The average second round upsets: ', upsets["SECOND ROUND"].mean().round(2))
print('The average sweet 16 upsets: ', upsets["SWEET 16"].mean())
print('The average elite 8 upsets: ', upsets["ELITE 8"].mean())
print('The average final 4 upsets: ', upsets["FINAL 4"].mean())
print('The average total tournament upsets: ', upsets["TOTAL"].mean().round(2))

The average first round upsets:  7.13
The average second round upsets:  3.33
The average sweet 16 upsets:  2.0
The average elite 8 upsets:  1.2
The average final 4 upsets:  0.2
The average total tournament upsets:  13.87


In [15]:
print('Misc stats')
print('There have always been at least ', upsets["FIRST ROUND"].min(), ' upsets in the first round, but there has not been more than ', upsets["FIRST ROUND"].max())
print('There are 2 or more upsets in the second round ', (round(len(upsets[upsets["SECOND ROUND"] >= 2]) / len(upsets["SECOND ROUND"]) * 100, 2)), '% of the time')
print('There are 2-4 upsets in the sweet 16 ', (round(len(upsets[upsets["SWEET 16"] >= 2]) / len(upsets["SWEET 16"]) * 100, 2)), '% of the time')

Misc stats
There have always been at least  5.0  upsets in the first round, but there has not been more than  10.0
There are 2 or more upsets in the second round  86.67 % of the time
There are 2-4 upsets in the sweet 16  66.67 % of the time


In [16]:
print('There is not a distinguishably noticable trend for the total upsets in the tournament as shown in this chart:')
Chart(upsets).mark_bar().encode(x='YEAR', y='TOTAL')

There is not a distinguishably noticable trend for the total upsets in the tournament as shown in this chart:


alt.Chart(...)

In [17]:
alt.Chart(upsets).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='YEAR:N'      # sort the colors of the circles by year
).properties(
    width=150,
    height=150
).repeat(     # make scatter plots for each comparing each data column to every other column
    row=['FIRST ROUND', 'SECOND ROUND', 'SWEET 16', 'ELITE 8', 'FINAL 4', 'TOTAL'],
    column=['FIRST ROUND', 'SECOND ROUND', 'SWEET 16', 'ELITE 8', 'FINAL 4', 'TOTAL']
).interactive()

alt.RepeatChart(...)

Using the correlation charts above, I can conclude that the columns with the largest correlation are the second round with the total upsets, and the sweet 16 with the total upsets. Both of these chars have a clear trend line that the chart follows, it is clear in these that the more upsets in these two rounds equates to the higher total upset years. The first round upsets with the total upsets had way too many outliers and plot points that didn't follow a trend.

-------------------------------

# CONFERENCE DATA

In [18]:
# group the conference data sheet by conference and only keep the columns we want to sum (amount of times conferences have made it to certain stages of the tourney)
group_by_conf = conf_data.groupby('CONFERENCE')[['CHAMP COUNT', 'FINALS COUNT', 'FINAL 4 COUNT', 'ELITE 8 COUNT', 'SWEET 16 COUNT']].sum().reset_index()
print('From 2008-2022, here are the number of teams that have made it to each stage of the tournament grouped by conference:')
group_by_conf

From 2008-2022, here are the number of teams that have made it to each stage of the tournament grouped by conference:


,CONFERENCE,CHAMP COUNT,FINALS COUNT,FINAL 4 COUNT,ELITE 8 COUNT,SWEET 16 COUNT
0,A10,0,0,0,2,7
1,ACC,5,7,10,20,37
2,AMER,1,1,2,2,4
3,B10,0,4,9,14,36
4,B12,3,5,7,17,30
5,BE,4,4,10,16,27
6,CUSA,0,1,1,1,2
7,HORZ,0,2,2,2,2
8,MVC,0,0,2,2,4
9,MWC,0,0,0,0,4


Key Takeaways:

- The winning team has always been in one of these five conferences: ACC (Atlantic Coast Conference), AMER (American Athletic Conference), B12 (Big 12), BE (Big East), and SEC (Southeastern Conference)
- The conference with the most winning ratio in the finals is the Big East, a team from the BE has made it to the finals 4 times and they have won the title all four times.
- The coference with the most losing ratio in the finals is the Big 10, the B10 has made it to the title game 4 times and has lost every single time.
- The Mountain West Conference (MWC) has not made it past the sweet 16.
- Despite having 19 appearances in the sweet 16, the PAC-12 (P12) have not made it to a finals game before.
- Among the conferences with a high number of sweet 16 appearances (15+), the ACC, BE, and SEC have the best conversion rates going into the elite 8, with the ACC moving on to the elite 8 54% of the time, the SEC moving on 68% of the time, and the BE moving on 59% of the time. The conferences with the worst conversion rate are the B10 and P12. The Big 10 has only moved on to the elite 8 39% of the time and the P12 has moved on only 37% of the time.

--------------------------------------

# COMPARING TO THE MOST RECENT TOURNAMENT (2023)

- The winner of this year's tournament (UConn) was a four seed heading into the tournament. In the tournaments history, UConn is the second ever four seed to win the championship.
- Despite having some of the most insane upset runs and one of the lowest average final four seed totals ever, this year's tournament rounds out the bottom edge of total first round upsets (since 2008) with only 5. 
- I found the stats following the upsets by round of the tournament in comparison with this year to be the most interesting. With five first round upsets, 2023 is below the average of 7.13 first round upsets. In contrast, 2023 is larger than the average by a considerable amount in terms of upsets for the rounds of the second round, sweet 16, and elite 8.
     - Average for second round: 3.33   || 2023 second round upsets: 4
     - Average for sweet 16: 2          || 2023 sweet 16 upsets: 4
     - Average for elite 8: 1.2         || 2023 elite 8 upsets: 2
- With all of that in consideration, 2023 ended up higher than average in terms of total upsets at 15 compared to the average of 13.87.
- (since 2008) Continuing the memorable considerations of 2023, this year's tournament shook up the conference data as well. With UConn coming out on top, nothing notable changes, the highly successfule Big East (BE) gain another champion and now tie the ACC for most wins. Moving past UConn, the championship losing team was 5 seed San Diego State and the rest of the final four consisted of 9 seed FAU and 5 seed Miami.
     - San Diego State made history becoming the first Mountain West Conference (MWC) team to make it to the championship game. In fact, a MWC team has never made it past the sweet 16 before San Diego State's run this year!
     - 9 seed Florida Atlantic University (FAU) became the second ever Conference USA (CUSA) team to make it past the sweet 16.
     - Miami did not make notable differences in the conference data as they are an ACC team.
     
     
The unpredictable but exciting behavior of this tournament is what I love about it. Having unknown teams/schools show up and make deep runs or even just one big upset allows under the radar teams to shine on the biggest stage!

--------------------------------------

# TLDR Takeaways

When picking a winner, try to stay away from teams with stats lower than these:

- KENPOM ADJUSTED EFFICIENCY:  19.1
- KENPOM ADJUSTED OFFENSE:  111.4
- KENPOM ADJUSTED DEFENSE:  83.2
- KENPOM ADJUSTED TEMPO:  59.3
- BARTTORVIK ADJUSTED EFFICIENCY:  19.0
- BARTTORVIK ADJUSTED OFFENSE:  111.6
- 2PT % DEFENSE:  38.8
- 3PT % DEFENSE:  27.2
- TURNOVER % DEFENSE:  17.8
- WIN %:  74.29
- POINTS PER POSSESSION OFFENSE:  1.085ENSE:  1.085

# Winner Seedings

- 1 seeds have the highest chance of winning (64.9%), followed by 2 and 3 seeds with similar win percentages.
- 5 seeds have never won the tournament, and 4, 6, 7, and 8 seeds have won once each.

# Upset Trends

- In about 3 out of 4 years there will be a 4/13 upset
- There is an EXTREMELY LIKELY chance that a 6/11 and 7/10 upset will happen, there has been at least one of each in every tournament for the last 15 years
- More often than not, there are 2 or more 6/11 upsets in the same year Elite 8.

# Conference Trends

- Since 2008, winning teams have only come from the ACC, AMER, B12, BE, and SEC conferences.
- The Big East has the best finals winning ratio (4/4), while the Big 10 has the worst (0/4).
- The PAC-12 has 19 sweet 16 appearances but has never reached the finals.

--------------------------------------

# Conclusion

Unlocking the secrets to March Madness success involves delving into the intricacies of team statistics, season to season variability, seeding odds, upset trends, conference performances, and luck. Teams boasting robust overall efficiency, favorable seeding, and a knack for upsetting specific matchups take center stage in the prediction game. Keeping a keen eye on how conferences perform and convert in later rounds adds an extra layer of precision to your predictive playbook. So, gear up, analyze wisely, and let the madness unfold!

--------------------------------------

# Updates

I started this project in the middle of March Madness 2023. I plan to update thi project each year with new findings/trend changes, plus the outcomes of my brackets and what worked/didn't work from following the guidelines found by using this analysis!

-------------------------------------------

# 2024

I did win both of my bracket challenges this year by using this project as an aide! Most notably, I chose the correct tournament winner pick of UConn using this analysis. I am planning to make quite a few changes/additions here this year! I will list some down below:
- Update all data with 2024 info
- Modify some of the hard-coded/printed analysis in this notebook so that it will auto update with the new CSV cells
- Implement a sci-kit learn pipeline
- Add an acc gradien boosting algorithm (XGBoost)
- Learn to use docker with this
- Add relevant spreadsheets to GitHub
- Add a requirements.txt file